## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [7]:
"""Code Here
    建立你的神經網路
    """
from keras.layers import BatchNormalization

def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [22]:
LEARNING_RATE = 1e-3
EPOCHS = 25 # IF you feel too run to finish, try to make it smaller
BATCH_SIZE = 1024 

In [23]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
reduce_lr_factor = [0.2, 0.4, 0.6]
redice_lr_patient = [3, 5,10 ]

In [ ]:
import itertools
results = {}
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i, %s" % (i, reduce_factor, reduce_patient, str(optim)))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    
    """Code Here
    設定 reduce learning rate 的 callback function
    """
    reduce_lr = ReduceLROnPlateau(factor= reduce_factor, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience= reduce_patient, 
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("%s,%s,rf=%s,rp=%s " % (i,str(optim),reduce_factor, reduce_patient))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, reduce_factor: 0.20, reduce_patient: 3, <keras.optimizers.SGD object at 0xb4013c1d0>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_60 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_178 (Bat (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_normalization_179 (Bat (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 256)               65792     
______________________________________

50000/50000 [==============================] - 5s 105us/step - loss: 1.5794 - acc: 0.4465 - val_loss: 1.6734 - val_acc: 0.4165
Epoch 13/25
50000/50000 [==============================] - 6s 121us/step - loss: 1.5608 - acc: 0.4534 - val_loss: 1.6609 - val_acc: 0.4226
Epoch 14/25
50000/50000 [==============================] - 7s 137us/step - loss: 1.5438 - acc: 0.4588 - val_loss: 1.6487 - val_acc: 0.4237
Epoch 15/25
50000/50000 [==============================] - 6s 118us/step - loss: 1.5286 - acc: 0.4642 - val_loss: 1.6449 - val_acc: 0.4263
Epoch 16/25
50000/50000 [==============================] - 6s 115us/step - loss: 1.5139 - acc: 0.4707 - val_loss: 1.6297 - val_acc: 0.4281
Epoch 17/25
50000/50000 [==============================] - 5s 108us/step - loss: 1.4997 - acc: 0.4732 - val_loss: 1.6249 - val_acc: 0.4334: 0s - loss: 1.4999 - acc:
Epoch 18/25
50000/50000 [==============================] - 6s 114us/step - loss: 1.4877 - acc: 0.4802 - val_loss: 1.6158 - val_acc: 0.4327
Epoch 19/25
5

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 15s 309us/step - loss: 2.5528 - acc: 0.1821 - val_loss: 2.4844 - val_acc: 0.2404
Epoch 2/25
50000/50000 [==============================] - 4s 89us/step - loss: 2.1040 - acc: 0.2862 - val_loss: 2.1132 - val_acc: 0.2940
Epoch 3/25
50000/50000 [==============================] - 6s 121us/step - loss: 1.9406 - acc: 0.3314 - val_loss: 1.9744 - val_acc: 0.3217
Epoch 4/25
50000/50000 [==============================] - 4s 85us/step - loss: 1.8537 - acc: 0.3568 - val_loss: 1.8835 - val_acc: 0.3462
Epoch 5/25
50000/50000 [==============================] - 5s 99us/step - loss: 1.7951 - acc: 0.3748 - val_loss: 1.8243 - val_acc: 0.3626
Epoch 6/25
50000/50000 [==============================] - 5s 94us/step - loss: 1.7506 - acc: 0.3880 - val_loss: 1.7979 - val_acc: 0.3724
Epoch 7/25
50000/50000 [==============================] - 4s 90us/step - loss: 1.7128 - acc: 0.4013 - val_loss: 1.7692 - val_

Epoch 24/25
50000/50000 [==============================] - 4s 88us/step - loss: 1.4334 - acc: 0.5011 - val_loss: 1.5827 - val_acc: 0.4472
Epoch 25/25
50000/50000 [==============================] - 4s 88us/step - loss: 1.4239 - acc: 0.5055 - val_loss: 1.5771 - val_acc: 0.4490
Numbers of exp: 5, reduce_factor: 0.40, reduce_patient: 10, <keras.optimizers.SGD object at 0xb4013c1d0>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_65 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_193 (Bat (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               65792     
_________________________

Epoch 11/25
50000/50000 [==============================] - 5s 93us/step - loss: 1.6003 - acc: 0.4408 - val_loss: 1.6832 - val_acc: 0.4093
Epoch 12/25
50000/50000 [==============================] - 6s 126us/step - loss: 1.5815 - acc: 0.4484 - val_loss: 1.6683 - val_acc: 0.4151
Epoch 13/25
50000/50000 [==============================] - 6s 111us/step - loss: 1.5635 - acc: 0.4541 - val_loss: 1.6559 - val_acc: 0.4186
Epoch 14/25
50000/50000 [==============================] - 5s 90us/step - loss: 1.5467 - acc: 0.4610 - val_loss: 1.6457 - val_acc: 0.4223
Epoch 15/25
50000/50000 [==============================] - 4s 89us/step - loss: 1.5328 - acc: 0.4654 - val_loss: 1.6308 - val_acc: 0.4254
Epoch 16/25
50000/50000 [==============================] - 4s 89us/step - loss: 1.5180 - acc: 0.4710 - val_loss: 1.6244 - val_acc: 0.4278
Epoch 17/25
50000/50000 [==============================] - 4s 89us/step - loss: 1.5045 - acc: 0.4753 - val_loss: 1.6188 - val_acc: 0.4319
Epoch 18/25
50000/50000 [=======

...

50000/50000 [==============================] - 4s 83us/step - loss: 2.0156 - acc: 0.3099 - val_loss: 2.0577 - val_acc: 0.3005
Epoch 22/25
50000/50000 [==============================] - 4s 86us/step - loss: 2.0013 - acc: 0.3138 - val_loss: 2.0447 - val_acc: 0.3051
Epoch 23/25
50000/50000 [==============================] - 5s 96us/step - loss: 1.9872 - acc: 0.3181 - val_loss: 2.0319 - val_acc: 0.3098
Epoch 24/25
50000/50000 [==============================] - 5s 104us/step - loss: 1.9749 - acc: 0.3230 - val_loss: 2.0210 - val_acc: 0.3126
Epoch 25/25
50000/50000 [==============================] - 5s 102us/step - loss: 1.9634 - acc: 0.3240 - val_loss: 2.0095 - val_acc: 0.3143
Numbers of exp: 20, reduce_factor: 0.20, reduce_patient: 10, <keras.optimizers.RMSprop object at 0xbf3898438>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_80 (InputLayer)        (None, 3072)              0         
____________

Epoch 8/25
50000/50000 [==============================] - 5s 104us/step - loss: 2.3368 - acc: 0.2346 - val_loss: 2.3146 - val_acc: 0.2481
Epoch 9/25
50000/50000 [==============================] - 5s 94us/step - loss: 2.2967 - acc: 0.2431 - val_loss: 2.2808 - val_acc: 0.2575
Epoch 10/25
50000/50000 [==============================] - 5s 100us/step - loss: 2.2621 - acc: 0.2516 - val_loss: 2.2496 - val_acc: 0.2637
Epoch 11/25
50000/50000 [==============================] - 5s 109us/step - loss: 2.2299 - acc: 0.2596 - val_loss: 2.2212 - val_acc: 0.2710
Epoch 12/25
50000/50000 [==============================] - 6s 118us/step - loss: 2.2005 - acc: 0.2677 - val_loss: 2.1946 - val_acc: 0.2753
Epoch 13/25
50000/50000 [==============================] - 5s 103us/step - loss: 2.1743 - acc: 0.2738 - val_loss: 2.1724 - val_acc: 0.2797
Epoch 14/25
50000/50000 [==============================] - 6s 114us/step - loss: 2.1488 - acc: 0.2804 - val_loss: 2.1500 - val_acc: 0.2846
Epoch 15/25
50000/50000 [=====

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 23s 451us/step - loss: 2.9101 - acc: 0.1029 - val_loss: 2.8193 - val_acc: 0.1228
Epoch 2/25
50000/50000 [==============================] - 5s 92us/step - loss: 2.7651 - acc: 0.1271 - val_loss: 2.7127 - val_acc: 0.1496
Epoch 3/25
50000/50000 [==============================] - 5s 93us/step - loss: 2.6514 - acc: 0.1517 - val_loss: 2.6297 - val_acc: 0.1726
Epoch 4/25
50000/50000 [==============================] - 5s 94us/step - loss: 2.5604 - acc: 0.1724 - val_loss: 2.5539 - val_acc: 0.1871
Epoch 5/25
50000/50000 [==============================] - 5s 99us/step - loss: 2.4856 - acc: 0.1917 - val_loss: 2.4910 - val_acc: 0.2002
Epoch 6/25
50000/50000 [==============================] - 5s 107us/step - loss: 2.4222 - acc: 0.2083 - val_loss: 2.4333 - val_acc: 0.2116
Epoch 7/25
50000/50000 [==============================] - 5s 92us/step - loss: 2.3686 - acc: 0.2217 - val_loss: 2.3827 - val_

50000/50000 [==============================] - 6s 119us/step - loss: 1.9923 - acc: 0.3189 - val_loss: 2.0304 - val_acc: 0.3159
Epoch 24/25
50000/50000 [==============================] - 4s 87us/step - loss: 1.9803 - acc: 0.3240 - val_loss: 2.0185 - val_acc: 0.3179
Epoch 25/25
50000/50000 [==============================] - 5s 90us/step - loss: 1.9685 - acc: 0.3271 - val_loss: 2.0067 - val_acc: 0.3194
Numbers of exp: 25, reduce_factor: 0.60, reduce_patient: 5, <keras.optimizers.RMSprop object at 0xbf3898438>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_85 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_253 (Bat (None, 256)               1024      
__________________________

50000/50000 [==============================] - 6s 110us/step - loss: 2.3804 - acc: 0.2336 - val_loss: 2.3937 - val_acc: 0.2426
Epoch 10/25
50000/50000 [==============================] - 6s 129us/step - loss: 2.3386 - acc: 0.2429 - val_loss: 2.3529 - val_acc: 0.2486
Epoch 11/25
50000/50000 [==============================] - 6s 113us/step - loss: 2.3019 - acc: 0.2492 - val_loss: 2.3180 - val_acc: 0.2552
Epoch 12/25
50000/50000 [==============================] - 7s 130us/step - loss: 2.2682 - acc: 0.2576 - val_loss: 2.2852 - val_acc: 0.2615
Epoch 13/25
50000/50000 [==============================] - 8s 150us/step - loss: 2.2374 - acc: 0.2643 - val_loss: 2.2572 - val_acc: 0.2696
Epoch 14/25
50000/50000 [==============================] - 6s 118us/step - loss: 2.2085 - acc: 0.2699 - val_loss: 2.2324 - val_acc: 0.2748
Epoch 15/25
50000/50000 [==============================] - 5s 100us/step - loss: 2.1820 - acc: 0.2759 - val_loss: 2.2100 - val_acc: 0.2768
Epoch 16/25
50000/50000 [==============

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(20,8))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(20,8))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()